Note: This python notebook requires the latest version of nltk (3.4.5). Kindly ensure only the latest version is imported

In [3]:
!pip3 install nltk==3.4.5

import re
import io
import numpy as np
import nltk
import math
from urllib import request

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##Getting Data from Project Gutengerg

In [0]:
url = "http://www.gutenberg.org/files/31100/31100.txt"
response = request.urlopen(url)
dataset = response.read().decode('utf8', 'replace')


##Sentence and Word Tokeniser

Note: 'train' and 'test' is refreshed and earsed everytime it is used in a computation. Thias is a bug in the latest nltk version (3.4.5) which has not been fixed. Hence before using the following blocks, re-run this block to re-initialise 'train' and 'test' - MLEdist_unigram block, MLEdist_bigram block, MLEdist_trigram block, MLEdist_quadgram block and perplexity() function 

In [0]:
sent_tokenize_list = nltk.tokenize.sent_tokenize(dataset[1291:])
for sent in sent_tokenize_list:
  sent_tokenize_list[sent_tokenize_list.index(sent)] = nltk.lm.preprocessing.pad_both_ends(list(nltk.tokenize.word_tokenize(re.sub(r'[^\w\s]\s|"\n"',' ',sent[0:len(sent)-1]))), n=2)

train = sent_tokenize_list[0:round( len(sent_tokenize_list)*0.8 )]
test = sent_tokenize_list[round( len(sent_tokenize_list)*0.8 ):]

##MLEdist_unigram Block

In [58]:
text_unigrams = [nltk.ngrams(sent, 1) for sent in train]
unigrams = []

for unigram in text_unigrams: unigrams = unigrams + list(unigram)
dist_unigrams = list(set(unigrams))  
  
print(dist_unigrams[0:10])

Fdist_unigram = nltk.probability.FreqDist(unigrams)
MLEdist_unigram = nltk.probability.MLEProbDist(Fdist_unigram)


[('raised',), ('cast',), ('survivor',), ('chanced',), ('creditable',), ('sides',), ('kindred',), ('ebony',), ('delusion',), ('self-approbation',)]


##MLEdist_bigram Block

In [27]:
text_bigrams = [nltk.ngrams(sent, 2) for sent in train]
bigrams = []

for bigram in text_bigrams: bigrams = bigrams + list(bigram)
dist_bigrams = list(set(bigrams))  
  
print(dist_bigrams[0:10])

Fdist_bigram = nltk.probability.FreqDist(bigrams)
MLEdist_bigram = nltk.probability.MLEProbDist(Fdist_bigram)

[('such', 'denial'), ('following', 'Pages'), ('than', 'ever'), ('cousin', 'Edmund'), ('ordered', 'a'), ('Emma', 'persevered'), ('Bertram', 'nor'), ('exact', 'steps'), ('other', 'night'), ('discovers', 'Popgun')]


##MLEdist_trigram Block

In [29]:
text_trigrams = [nltk.ngrams(sent, 3) for sent in train]
trigrams = []

for trigram in text_trigrams: trigrams = trigrams + list(trigram)
dist_trigrams = list(set(trigrams))  
  
print(dist_trigrams[0:10])

Fdist_trigram = nltk.probability.FreqDist(trigrams)
MLEdist_trigram = nltk.probability.MLEProbDist(Fdist_trigram)


[('desire', 'of', 'seeing'), ('her', 'from', 'Portsmouth'), ('privileges', 'to', 'be'), ('dear', 'I', 'should'), ('that', 'way', 'most'), ('of', 'little', 'consequence'), ('the', 'same', 'thing'), ('She', 'was', 'willing'), ('often', 'wrong', '.'), ('dangerous', 'precedent', 'indeed')]


##MLEdist_quadgram Block

In [0]:
text_quadgrams = [nltk.ngrams(sent, 4) for sent in train]
quadgrams = []

for quadgram in text_quadgrams: quadgrams = quadgrams + list(quadgram)
dist_quadgrams = list(set(quadgrams))

print(dist_quadgrams[0:10])

Fdist_quadgram = nltk.probability.FreqDist(quadgrams)
MLEdist_quadgram = nltk.probability.MLEProbDist(Fdist_quadgram)

##Generators Functions

In [0]:
def Generator_unigram() :
  sum_MLE = 0.0
  norm_MLE_list = list()
  norm_MLE_word = list()
  
  for k,v in Fdist_unigram.items():
    sum_MLE = sum_MLE+ MLEdist_unigram.prob(k)
  for k,v in Fdist_unigram.items():
    norm_MLE_list.append(MLEdist_unigram.prob(k)/sum_MLE)
    norm_MLE_word.append(k)
  
  index = np.where(np.random.multinomial(1, norm_MLE_list, size=1) == 1)
  return norm_MLE_word[index[1][0]][0]
  



In [0]:
def Generator_bigram(start) :
  sum_MLE = 0.0
  norm_MLE_list = list()
  norm_MLE_word = list()
  
  for k,v in Fdist_bigram.items():
    if start == k[0]:
      sum_MLE = sum_MLE + MLEdist_bigram.prob(k)
  for k,v in Fdist_bigram.items():
    if start == k[0]:
      norm_MLE_list.append(MLEdist_bigram.prob(k)/sum_MLE)
      norm_MLE_word.append(k)
     
    
  index = np.where(np.random.multinomial(1, norm_MLE_list, size=1) == 1)
  return norm_MLE_word[index[1][0]][1]


In [0]:
def Generator_trigram(start1,start2) :
    sum_MLE = 0.0
    norm_MLE_list = list()
    norm_MLE_word = list()
    word2 = str()
    
    for k,v in Fdist_bigram.items():
        if start1 == "<s>" and start2 =="<s>":
            word2 = Generator_bigram(start1)
            if start1 == k[0] and word2 == k[1]:
                sum_MLE = sum_MLE + MLEdist_trigram.prob(k)
        elif start1 == k[0] and start2 == k[1]:
            sum_MLE = sum_MLE + MLEdist_trigram.prob(k)
    for k,v in Fdist_bigram.items():
        if start1 == "<s>" and start2 =="<s>":
            if start1 == k[0] and word2 == k[1]:
                norm_MLE_list.append(MLEdist_trigram.prob(k)/sum_MLE)
                norm_MLE_word.append(k)
        elif start1 == k[0] and start2 == k[1]:
            norm_MLE_list.append(MLEdist_trigram.prob(k)/sum_MLE)
            norm_MLE_word.append(k)
            
    index = np.where(np.random.multinomial(1, norm_MLE_list, size=1) == 1)
    return norm_MLE_word[index[1][0]][2]
  

In [0]:
def Generator_quadgram(start1,start2,start3) :
    sum_MLE = 0.0
    norm_MLE_list = list()
    norm_MLE_word = list()
    word2 = str()
    word3 = str()
    
    for k,v in Fdist_bigram.items():
        if start1 == "<s>" and start2 =="<s>" and start3 =="<s>":
            word2 = Generator_bigram(start1)
            word3 = Generator_trigram(start1,word2)
            if start1 == k[0] and word2 == k[1] and word3 == k[2]:
                sum_MLE = sum_MLE + MLEdist_quadgram.prob(k)
        elif start1 == k[0] and start2 == k[1] and start2 == k[1]:
            sum_MLE = sum_MLE + MLEdist_quadgram.prob(k)
    for k,v in Fdist_bigram.items():
        if start1 == "<s>" and start2 =="<s>" and start3 =="<s>":
              if start1 == k[0] and word2 == k[1] and word3 == k[2]:
                    norm_MLE_list.append(MLEdist_quadgram.prob(k)/sum_MLE)
                    norm_MLE_word.append(k)
        elif start1 == k[0] and start2 == k[1]:
            norm_MLE_list.append(MLEdist_quadgram.prob(k)/sum_MLE)
            norm_MLE_word.append(k)
    
    index = np.where(np.random.multinomial(1, norm_MLE_list, size=1) == 1)
    return norm_MLE_word[index[1][0]][2]

##Random Sentence Generators using Generators Functions
Note: trigram and quadgram take very long to compute and hence when using on colab, the block sometimes times-out 

In [0]:
ngram = str()
word = "<s>"
sentence = list()
space = " "

print("Chose N-Gram: ")
ngram = input()

while(word!="</s>"):
    sentence.append(word)
    if ngram == "unigram":
        word = Generator_unigram()
    elif ngram == "bigram":
        word = Generator_bigram(sentence[-1])
    elif ngram == "trigram":
        if word=="<s>" and len(sentence)==1:
            word = Generator_trigram(word,word)
        else:
            word = Generator_trigram(sentence[-2],sentence[-1])
    elif ngram == "quadgram":
        if word=="<s>" and len(sentence)==1:
            word = Generator_quadgram(word,word,word)
        else:
            word = Generator_quadgram(sentence[-3],sentence[-2],sentence[-1])
    print(space.join(sentence))

##Perplexity Function 
Note: Written strictly as per defination of function

In [0]:
def perplexity():
    print("Input Test Sentence Sequence - Number in Corpus - you Want to test perplexity of")  
    s = int(input())
    print("Input N-Gram Model you want to use to derive probabilites")  
    ngram = input()
    
    log_add = 0.0
    train_p = list(train[s])
    #print(train_p)
    
    if ngram == "unigram":
        grams = [nltk.ngrams(train_p, 1)]
    elif ngram == "bigram":
        grams = [nltk.ngrams(train_p, 2)]
    elif ngram == "trigram":
        grams = [nltk.ngrams(train_p, 3)]
    elif ngram == "quadgram":
        grams = [nltk.ngrams(train_p, 4)]
        
    grams_list = []
    
    for gram in grams: grams_list = grams_list + list(gram)
        
    if ngram == "unigram":
        for k,v in Fdist_unigram.items():
            if k in grams_list:
                log_add = math.log(MLEdist_unigram.prob(k),2)
    elif ngram == "bigram":
        for k,v in Fdist_bigram.items():
            if k in grams_list:
                log_add = math.log(MLEdist_bigram.prob(k),2)
    elif ngram == "trigram":
        for k,v in Fdist_trigram.items():
            if k in grams_list:
                log_add = math.log(MLEdist_trigram.prob(k),2)
    elif ngram == "quadgram":
        for k,v in Fdist_quadgram.items():
            if k in grams_list:
                log_add = math.log(MLEdist_quadgram.prob(k),2)
    
    l = log_add/len(train_p)
    
    ppw = pow(2,-l)
    
    print("Perplexity is: ")
    return ppw


In [0]:
print(perplexity())